# Milan



### R Libraries

The relvant R libraries are imported in to the kernal:

In [1]:
# libraries
# Load R libraries
 if(!require("pacman"))
     install.packages("pacman")
     library("pacman")

p_load("dplyr", "sf", "purrr", "tidyverse")

print("Loaded Packages:")
p_loaded()

Loading required package: pacman



[1] "Loaded Packages:"


[1] "lubridate" "forcats"   "stringr"   "readr"     "tidyr"     "tibble"   
 [7] "ggplot2"   "tidyverse" "purrr"     "sf"        "dplyr"     "pacman"

In [2]:
# create the pipeline directory if it does not exist
pipelineDir <- file.path("../..","2_pipeline","Milan","1a_CensusData","2011")
if(!dir.exists(pipelineDir)){
    dir.create(pipelineDir, recursive = TRUE)
    print(paste0(pipelineDir, " created"))
}

### Import the csv data

Italy census data from: http://dati-censimentopopolazione.istat.it/Index.aspx?lang=en and https://www.istat.it/it/


In [3]:
# Characteristics of occupation
censusCARATT <- read.csv('../../0_data/Milan/ItalianCensus/2011/DICA_CARATT_ATTL_Data+FootnotesLegend_6172c195-7683-4a83-bf53-58e22f88399d.csv',
                         sep="|", fileEncoding="latin1")

# Current activity status of resident population
censusCONDPROF <- read.csv('../../0_data/Milan/ItalianCensus/2011/DICA_CONDPROF_Data+FootnotesLegend_960a1c2f-afb1-4c52-bc02-137cb8952b3d.csv',
                           sep="|", fileEncoding="latin1")

# Private households characteristics
censusFAM <- read.csv('../../0_data/Milan/ItalianCensus/2011/DICA_FAM_CARATT_Data+FootnotesLegend_be5873e0-9b10-4c3d-afde-6af25943f561.csv',
                      sep="|", fileEncoding="latin1")

# Educational attainement of resident population aged 6 years and over
censusGRADOISTR <- read.csv('../../0_data/Milan/ItalianCensus/2011/DICA_GRADOISTR_Data+FootnotesLegend_59e8872b-f081-49e3-a028-5af854a69add.csv',
                            sep="|", fileEncoding="latin1")

# Educational attainement rate
censusDICA <- read.csv ('../../0_data/Milan/ItalianCensus/2011/DICA_INDIC_EDU_Data+FootnotesLegend_3f2fc553-7377-429a-a309-7a003d598ad3.csv',
                        sep="|", fileEncoding="latin1")

# Current activity status rate
censusINDIC <- read.csv('../../0_data/Milan/ItalianCensus/2011/DICA_INDIC_LAV_Data+FootnotesLegend_18168aff-3f7c-4452-a29b-f561a8dfb655.csv',
                        sep="|")

# Conventional dwellings and buildings by type of locality
censusLCAB <- read.csv('../../0_data/Milan/ItalianCensus/2011/DICA_LCAB_AE_Data+FootnotesLegend_162ea98c-601a-4efc-ad71-294d67205444.csv',
                       sep="|")

# Commuting for studying or working
censusPEND <- read.csv('../../0_data/Milan/ItalianCensus/2011/DICA_PEND_Data+FootnotesLegend_dbd054e6-8464-4842-8e34-aa038c486721.csv',
                       sep="|", fileEncoding="latin1")

# Resident population
censusPOP <- read.csv('../../0_data/Milan/ItalianCensus/2011/DICA_POP_RESPRE_Data+FootnotesLegend_c313bc18-833e-46ed-98e1-bede3a0fb476.csv',
                      sep="|", fileEncoding="latin1")

# Resident population by citizenship 
censusPOPCIT <- read.csv('../../0_data/Milan/ItalianCensus/2011/DICA_POP_RSPR_CITT1_Data+FootnotesLegend_4701c6af-38c9-48a3-92db-8e795632e1bc.csv',
                         sep="|", fileEncoding="latin1")

# Population in institutional household
censusPOPINCV <- read.csv('../../0_data/Milan/ItalianCensus/2011/DICA_POPINCV_Data+FootnotesLegend_053202f0-3679-41a9-b6d1-4283da95d60e.csv',
                          sep="|", fileEncoding="latin1")

# Employed persons by industry (branch of economic activity) 
censusCARATT_ATTL_COM <- read.csv('../../0_data/Milan/ItalianCensus/2011/DICA_CARATT_ATTL_COM_Data+FootnotesLegend_52172124-ca1f-424d-87b3-ce67ec1fcc43.csv',
                                  sep="|", fileEncoding="latin1")

# Family nuclei
censusNUCLEI <- read.csv('../../0_data/Milan/ItalianCensus/2011/DICA_NUCLEI_Data+FootnotesLegend_c2a8cf1e-25a3-44f8-9c63-514772491d3e.csv',
                         sep="|", fileEncoding="latin1")

# households by family type
censusFAM_CARATT4 <- read.csv('../../0_data/Milan/ItalianCensus/2011/DICA_FAM_CARATT4_Data+FootnotesLegend_a860bec2-1750-4623-ba1b-2796de241047.csv',
                              sep="|", fileEncoding="latin1")

# showers/bathing
censusABIT <- read.csv('../../0_data/Milan/ItalianCensus/2011/DICA_ABIT_PR_COM_Data+FootnotesLegend_178e215f-c09a-4046-b67c-346c6d5c2f28.csv',
                       sep="|", fileEncoding="latin1")

# housing units
censusALLOGGI <- read.csv('../../0_data/Milan/ItalianCensus/2011/DICA_ALLOGGI_Data+FootnotesLegend_5e5ff20f-52be-43ae-88b8-8ab1b64ce175.csv',
                          sep="|", fileEncoding="latin1")

# building floors
censusEDIFICIRE <- read.csv('../../0_data/Milan/ItalianCensus/2011/DICA_EDIFICIRES_Data+FootnotesLegend_b0e5ecc6-780a-41e1-93f6-29d1c470cd57.csv',
                            sep="|")

censusData <- c(censusCARATT, censusCONDPROF, censusFAM, censusGRADOISTR, censusINDIC, censusLCAB, censusPEND, censusPOP,censusPOPCIT, censusPOPINCV, censusCARATT_ATTL_COM, censusNUCLEI, censusFAM_CARATT4, censusABIT, censusALLOGGI, censusEDIFICIRE)

In [4]:
# set the census year
censusYear <- 2011

### Import Census Output Areas 

The spatial output areas need to be ingested.

In [5]:
#set the path to the census OA directory
censusOAPath <- "../../0_data/Milan/OA/2011/Limiti2011/Com2011/Com2011_WGS84.shp"

#read the OA data
censusOA <- st_read(censusOAPath)

Reading layer `Com2011_WGS84' from data source 
  `/Volumes/2023_WD18TB/REACHOUT/teams/Cities/0_data/Milan/OA/2011/Limiti2011/Com2011/Com2011_WGS84.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 8092 features and 11 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 313279.3 ymin: 3933683 xmax: 1312016 ymax: 5220292
Projected CRS: WGS 84 / UTM zone 32N


In [6]:
#get a list of unique ID's
oaID <- censusOA$PRO_COM
length(oaID)

[1] 8092

## Select only the relevant data

We only require a subset of the census data for our purposes. We therefore need to extract the relevant data, then combine these to create our vulnerability indicators.


### Persons Level Data

We then get the persons level data and combine the variables together to create indicators:

In [7]:
#PERSONS DATA
censusPopData <- censusPOP[, c('ITTER107', 'Territory', 'Data.type', 'Gender','Age', 'Age.by.year.of.birth.and.completed.years.of.age', 'Legal.marital.status', 'Type.of.building', 'Census.year', 'Value'), drop = FALSE]
#POPULATION TOTAL
populationTotalData <- censusPopData %>%
        filter(Data.type == 'present population (absolute values)', 
              Gender == 'total',
              Age == 'total',
              Age.by.year.of.birth.and.completed.years.of.age == 'all items',
              Legal.marital.status == 'total',
              Type.of.building == 'all items',
              Census.year == censusYear
               )
populationTotalData <- populationTotalData[c("ITTER107","Territory","Value")]
colnames(populationTotalData) <- c("ITTER107","Territory","totalPresPop_abs")

# get rid of the larger areas 
populationTotalData$ITTER107 <- as.numeric(as.character(populationTotalData$ITTER107))
populationTotalData <- populationTotalData %>% filter(ITTER107 != "NA")


#AGE - YOUNG 
ageYoungData <- censusPopData %>%
        filter(Data.type == 'resident population (percentage values)', 
              Gender == 'total',
              Age == 'until 4 years',
              Legal.marital.status == 'total',
              Type.of.building == 'all items',
              Census.year == censusYear
               )
ageYoungData <- ageYoungData[c("ITTER107","Value")]
colnames(ageYoungData) <- c("ITTER107","ageYoung_perc")

# get rid of the larger areas 
ageYoungData$ITTER107 <- as.numeric(as.character(ageYoungData$ITTER107))
ageYoungData <- ageYoungData %>% filter(ITTER107 != "NA")


# ageYoungVariables <- ageYoungVariables %>% dplyr::filter(
#     grepl(paste0("\\b", 0:youngAgeMax, " years", collapse = "|"), Age.by.year.of.birth.and.completed.years.of.age))


#AGE - OLD 
ageOldData <- censusPopData %>%
        filter(Data.type == 'resident population (percentage values)', 
              Gender == 'total',
              Age == '85 years and over',
              Legal.marital.status == 'total',
              Type.of.building == 'all items',
              Census.year == 2011
               )
ageOldData <- ageOldData[c("ITTER107","Value")]
colnames(ageOldData) <- c("ITTER107","ageOld_perc")

# get rid of the larger areas 
ageOldData$ITTER107 <- as.numeric(as.character(ageOldData$ITTER107))
ageOldData <- ageOldData %>% filter(ITTER107 != "NA")


#POPULATION DENSITY
popDenData <- censusPopData %>%
        filter(Data.type == ' population density ', 
              Gender == 'total',
              Age == 'total',
              Age.by.year.of.birth.and.completed.years.of.age == 'all items',
              Legal.marital.status == 'total',
              Type.of.building == 'all items',
              Census.year == censusYear
               )
popDenData <- popDenData[c("ITTER107","Value")]
colnames(popDenData) <- c("ITTER107","popDen")

# get rid of the larger areas 
popDenData$ITTER107 <- as.numeric(as.character(popDenData$ITTER107))
popDenData <- popDenData %>% filter(ITTER107 != "NA")

#combine all the data into one table
personsData <- merge(x = populationTotalData, y = ageYoungData, by = 'ITTER107', all.x = TRUE)
personsData <- merge(x = personsData, y = ageOldData, by = 'ITTER107')
personsData <- merge(x = personsData, y = popDenData, by = 'ITTER107')

head(personsData)

#output the data as a csv
write.csv(personsData, file.path(pipelineDir, "persons_age_raw.csv"), row.names = FALSE)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


,ITTER107,Territory,totalPresPop_abs,ageYoung_perc,ageOld_perc,popDen
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,1001,Agliè,2608,4.08,4.01,201.12
2,1002,Airasca,3790,5.68,1.70,242.64
3,1003,Ala di Stura,435,3.03,1.73,9.97
4,1004,Albiano d'Ivrea,1847,3.91,3.07,152.66
5,1005,Alice Superiore,694,3.85,3.99,94.99
6,1006,Almese,6173,4.22,2.79,352.60


In [8]:
# FOREIGN CITIZENS
censusPOPCITData <- censusPOPCIT[, c('ITTER107', 
                                     'Territory', 
                                     'Type.of.territory',
                                     'Data.Type', 
                                     'Gender',
                                     'Age',
                                     'Legal.marital.status',
                                     'Citizenship', 
                                     'Geographical.area.of.citizenship', 
                                     'Place.of.birth', 
                                     'Reason.of.citizenship.acquisition',
                                     'Year.of.arrival.in.Italy',
                                     'Census.year',
                                     'Value')]

# get the foreign citizens
foreignCitData <- censusPOPCITData %>%
        filter(Type.of.territory == 'all items',
               Age == 'total',
               Legal.marital.status == 'total',
               Citizenship == 'foreign/stateless',
               Geographical.area.of.citizenship == 'all items',
               Place.of.birth == 'all items',
               Reason.of.citizenship.acquisition == 'all items',
               Year.of.arrival.in.Italy == 'all items',
               Data.Type == 'resident population (absolute values)', 
               Gender == 'total',
               Census.year == censusYear
               )
# summarise the data
foreignCitData <- foreignCitData[c("ITTER107","Value")]
colnames(foreignCitData) <- c("ITTER107","foreignCit_abs")

# get rid of the larger areas
foreignCitData$ITTER107 <- as.numeric(as.character(foreignCitData$ITTER107))


# get the total number of citizens
totalCitData <- censusPOPCITData %>%
        filter(Type.of.territory == 'all items',
               Age == 'total',
               Legal.marital.status == 'total',
               Citizenship == 'total',
               Geographical.area.of.citizenship == 'all items',
               Place.of.birth == 'all items',
               Reason.of.citizenship.acquisition == 'all items',
               Year.of.arrival.in.Italy == 'all items',
               Data.Type == 'resident population (absolute values)', 
               Gender == 'total',
               Census.year == censusYear
               )
# summarise the data
totalCitData <- totalCitData[c("ITTER107","Value")]
colnames(totalCitData) <- c("ITTER107","totalCit_abs")

# get rid of the larger areas
totalCitData$ITTER107 <- as.numeric(as.character(totalCitData$ITTER107))
totalCitData <- na.omit(totalCitData)


# merge the data 
foreignCitData <- merge(foreignCitData,totalCitData, by='ITTER107')

# calculate the foreign cit percentage
foreignCitData$foreignCit_perc <- round((foreignCitData$foreignCit_abs/foreignCitData$totalCit_abs)*100, 2)

foreignCitData <- foreignCitData[c("ITTER107","foreignCit_perc")]

##output the data as a csv
#write.csv(foreignCitData, file.path(pipelineDir, "persons_foreign_raw.csv"), row.names = FALSE)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [9]:
# Educational attainment status rate MetaData : Compulsory school non attainment rate by gender - DICA_INDIC_EDU - Educational attainement status rate - Full dataset	
censusDICAData <- censusDICA[, c('ITTER107', 
                                    'Territory', 
                                    'Data.Type', 
                                    'Gender',
                                    'Age',
                                    'Census.year',
                                    'Value')]

eduAttainData <- censusDICAData %>%
        filter(
               Age == '15-62 anni',
               Data.Type == 'compulsory school non attainment rate', 
               Gender == 'total',
               Census.year == censusYear
               )

eduAttainData <- eduAttainData[c("ITTER107","Value")]
colnames(eduAttainData) <- c("ITTER107","eduAttain_perc")

# get rid of the larger areas
eduAttainData$ITTER107 <- as.numeric(as.character(eduAttainData$ITTER107))


Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [10]:
# JOBS/WORK number of unemployed in economically active population
censusCONDPROFData <- censusCONDPROF[, c('ITTER107', 
                                    'Territory',
                                    'Gender',
                                    'Age',
                                    'Legal.marital.status',
                                    'Citizenship',
                                    'Geographical.area.of.citizenship',
                                    'Educational.attainment',
                                    'Attendance.to.one.or.more.vocational.trainings',
                                    'Type.of.institutional.household',                                   
                                    'Census.year',
                                    'Data.Type', 
                                    'Current.activity.status',
                                    'Value')]

# get the total number of economic active
economicActiveData <- censusCONDPROFData %>%
        filter(
               Gender == 'total',
               Age == '15 years and over',
               Legal.marital.status == 'total',
               Citizenship == 'total',
               Geographical.area.of.citizenship == 'all items',
               Educational.attainment == 'total',
               Attendance.to.one.or.more.vocational.trainings == 'all items',
               Type.of.institutional.household == 'all items',
               Census.year == censusYear,
               Data.Type == 'resident population (absolute values)',
               Current.activity.status == 'currently economically active'
               )
economicActiveData <- economicActiveData[c("ITTER107","Value")]
colnames(economicActiveData) <- c("ITTER107","economicActive_abs")

# get rid of the larger areas
economicActiveData$ITTER107 <- as.numeric(as.character(economicActiveData$ITTER107))
economicActiveData <- na.omit(economicActiveData)


# get the total number of unemployed 
unemployedData <- censusCONDPROFData %>%
        filter(
               Gender == 'total',
               Age == '15 years and over',
               Legal.marital.status == 'total',
               Citizenship == 'total',
               Geographical.area.of.citizenship == 'all items',
               Educational.attainment == 'total',
               Attendance.to.one.or.more.vocational.trainings == 'all items',
               Type.of.institutional.household == 'all items',
               Census.year == censusYear,
               Data.Type == 'resident population (absolute values)',
               Current.activity.status == 'unemployed person'
               )
unemployedData <- unemployedData[c("ITTER107","Value")]
colnames(unemployedData) <- c("ITTER107","unemployed_abs")

# get rid of the larger areas
unemployedData$ITTER107 <- as.numeric(as.character(unemployedData$ITTER107))
unemployedData <- na.omit(unemployedData)


# join the two tables
unemployedPerc <- merge(economicActiveData, unemployedData, by = 'ITTER107')

# calculate the unemployment percentage
unemployedPerc$unemployed_perc <- round((unemployedPerc$unemployed_abs/unemployedPerc$economicActive_abs)*100, 2)

# trim the data
unemployedPercData <- unemployedPerc[c("ITTER107","unemployed_perc")]

##output the data as a csv
#write.csv(unemployedPercData, file.path(pipelineDir, "persons_unemployed.csv"), row.names = FALSE)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [11]:
# type of work  Employed persons by industry (branch of economic activity) - Municipal data MetaData - DICA_CARATT_ATTL_COM - Employed persons by industry (branch of economic activity) - Municipal data - Full dataset
censusCARATT_ATTL_COMData <- censusCARATT_ATTL_COM[, c('ITTER107', 
                                    'Territory',
                                    'Gender', 
                                    'Census.year',
                                    'Data.Type', 
                                    'Industry..branch.of.economic.activity.',
                                    'Value')]

# get the total number employed
employedPersonsData <- censusCARATT_ATTL_COMData %>%
        filter(
               Gender == 'total',
               Census.year == censusYear,
               Data.Type == 'employed persons (average values)',
               Industry..branch.of.economic.activity. == 'total' 
               )

employedPersonsData <- employedPersonsData[c("ITTER107","Value")]
colnames(employedPersonsData) <- c("ITTER107","employed_abs")

# get rid of the larger areas
employedPersonsData$ITTER107 <- as.numeric(as.character(employedPersonsData$ITTER107))
employedPersonsData <- na.omit(employedPersonsData)


# get the number of people invovled in agriculature, forestry, fishing
employedAgForFisPersonsData <- censusCARATT_ATTL_COMData %>%
        filter(
               Gender == 'total',
               Census.year == censusYear,
               Data.Type == 'employed persons (average values)',
               Industry..branch.of.economic.activity. == 'agriculture, forestry and fishing' 
               )

employedAgForFisPersonsData <- employedAgForFisPersonsData[c("ITTER107","Value")]
colnames(employedAgForFisPersonsData) <- c("ITTER107","ag_for_fi_employed_abs")

# get rid of the larger areas
employedAgForFisPersonsData$ITTER107 <- as.numeric(as.character(employedAgForFisPersonsData$ITTER107))


# join the two tables
employedAgForFisPersonsPerc <- merge(employedPersonsData, employedAgForFisPersonsData, by = 'ITTER107')

# calculate the unemployment percentage
employedAgForFisPersonsPerc$ag_for_fi_employed_perc <- round((employedAgForFisPersonsPerc$ag_for_fi_employed_abs/employedAgForFisPersonsPerc$employed_abs)*100, 2)

# trim the data
employedAgForFisPersonsPerc <- employedAgForFisPersonsPerc[c("ITTER107","ag_for_fi_employed_perc")]

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


## Merge all Data

In [12]:
# Merge all the persons data into a single dataframe
personsDataAll <- merge(x = personsData, y = foreignCitData, by = 'ITTER107', all.x = TRUE)
personsDataAll <- merge(x = personsDataAll, y = eduAttainData, by = 'ITTER107', all.x = TRUE)
personsDataAll <- merge(x = personsDataAll, y = unemployedPercData, by = 'ITTER107', all.x = TRUE)
personsDataAll <- merge(x = personsDataAll, y = employedAgForFisPersonsPerc, by = 'ITTER107', all.x = TRUE)

#output the data as a csv
write.csv(personsDataAll, file.path(pipelineDir, "persons_data.csv"), row.names = FALSE)

# merge the census data with the spatial output area
oaMerge <- merge(censusOA, personsDataAll, by.x = 'PRO_COM', by.y = 'ITTER107', all.x = TRUE)

#output the data as a geojson
st_write(oaMerge, file.path(pipelineDir, "oa_persons.geojson"), delete_dsn = TRUE)

Deleting source `../../2_pipeline/Milan/1a_CensusData/2011/oa_persons.geojson' using driver `GeoJSON'
Writing layer `oa_persons' to data source 
  `../../2_pipeline/Milan/1a_CensusData/2011/oa_persons.geojson' using driver `GeoJSON'
Writing 8092 features with 20 fields and geometry type Multi Polygon.


### Household Level Data

We then get the household level data and combine the variables together to create indicators:

In [13]:
# renters
censusFAMData <- censusFAM[, c('ITTER107', 
                                'Territory',               
                                'Type.of.private.household',
                                'Gender.of.reference.person.of.the.household',
                                'Age.class.of.reference.person.of.the.household',
                                'Legal.marital.status.of.reference.person.of.the.household',
                                'Single.or.shared.occupancy',
                                'Housing.arrangements',
                                'Number.of.members.in.the.household',
                                'Type.of.living.quarter',
                                'Tenure.status.of.the.household',
                                'Census.year',
                                'Data.type', 
                                'Value')]

# get the total number of private households
totalHousholdsData <- censusFAMData %>%
        filter(
            Type.of.private.household == 'private households', 
            Gender.of.reference.person.of.the.household == 'total',
            Age.class.of.reference.person.of.the.household == 'total',
            Legal.marital.status.of.reference.person.of.the.household == 'total',
            Single.or.shared.occupancy == 'all items',
            Housing.arrangements == 'all items',
            Number.of.members.in.the.household == 'total',
            Type.of.living.quarter == 'all items',
            Tenure.status.of.the.household == 'all items',
            Census.year == censusYear,
            Data.type == 'number of private households (absolute values)'
               )

totalHousholdsData <- totalHousholdsData[c("ITTER107","Value")]
colnames(totalHousholdsData) <- c("ITTER107","total_households_abs")

# get rid of the larger areas
totalHousholdsData$ITTER107 <- as.numeric(as.character(totalHousholdsData$ITTER107))
totalHousholdsData <- na.omit(totalHousholdsData)


# get the total number of renting households
rentHousholdsData <- censusFAMData %>%
        filter(
            Type.of.private.household == 'private households', 
            Gender.of.reference.person.of.the.household == 'total',
            Age.class.of.reference.person.of.the.household == 'total',
            Legal.marital.status.of.reference.person.of.the.household == 'total',
            Single.or.shared.occupancy == 'all items',
            Housing.arrangements == 'all items',
            Number.of.members.in.the.household == 'total',
            Type.of.living.quarter == 'all items',
            Tenure.status.of.the.household == 'rent',
            Census.year == censusYear
               )

rentHousholdsData <- rentHousholdsData[c("ITTER107","Value")]
colnames(rentHousholdsData) <- c("ITTER107","rent_households_perc")

# get rid of the larger areas
rentHousholdsData$ITTER107 <- as.numeric(as.character(rentHousholdsData$ITTER107))
rentHousholdsData <- na.omit(rentHousholdsData)

# merge the data
rentHousholdsData <- merge(rentHousholdsData, totalHousholdsData, by = 'ITTER107')
# rentHousholdsData

# trim data
rentHousholdsData <- rentHousholdsData[c("ITTER107","rent_households_perc")]

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [14]:
# single parents
censusNUCLEIData <- censusNUCLEI[, c('ITTER107', 
                                'Territory',               
                                'Type.of.family.nucleus',
                                'Sex.of.the.parent',
                                'Age.class.of.the.parent',
                                'Legal.marital.status.of.the.parent',
                                'Number.of.children',
                                'Geographical.area.and.country.of.citizenship',
                                'Census.year',
                                'Data.Type', 
                                'Value')]

# get the total number of single mother households
singleMotherData <- censusNUCLEIData %>%
        filter(
        Type.of.family.nucleus == 'lone mother households',
        Sex.of.the.parent == "total", 
        Age.class.of.the.parent == "total", 
        Legal.marital.status.of.the.parent == "total", 
        Number.of.children == "total", 
        Geographical.area.and.country.of.citizenship == "all items", 
        Census.year == censusYear,
        Data.Type == 'number of family nuclei (absolute values)'
               )

singleMotherData <- singleMotherData[c("ITTER107","Value")]
colnames(singleMotherData) <- c("ITTER107","singleMothers_abs")

# get the total number of single mother households
singleFatherData <- censusNUCLEIData %>%
        filter(
        Type.of.family.nucleus == 'lone father households',
        Sex.of.the.parent == "total", 
        Age.class.of.the.parent == "total", 
        Legal.marital.status.of.the.parent == "total", 
        Number.of.children == "total", 
        Geographical.area.and.country.of.citizenship == "all items", 
        Census.year == censusYear,
        Data.Type == 'number of family nuclei (absolute values)'
               )

singleFatherData <- singleFatherData[c("ITTER107","Value")]
colnames(singleFatherData) <- c("ITTER107","singleFathers_abs")

# merge data
singleParentData <- merge(singleMotherData, singleFatherData, by = 'ITTER107')

# # total single parent
singleParentData$singleParent_abs <- singleParentData$singleMothers_abs + singleParentData$singleFathers_abs

# # # get rid of the larger areas
singleParentData$ITTER107 <- as.numeric(as.character(singleParentData$ITTER107))
singleParentData <- na.omit(singleParentData)

# # #get the total number of households and join
singleParentData <- merge(totalHousholdsData, singleParentData, by = 'ITTER107')

# # # perc single parent
singleParentData$singleParent_perc <- round((singleParentData$singleParent_abs/singleParentData$total_households_abs)*100, 2)

# # # trim data
singleParentData <- singleParentData[c("ITTER107","singleParent_perc")]

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [15]:
# sole occupants
censusFAMData <- censusFAM[, c('ITTER107', 
                                'Territory',               
                                'Age.class.of.reference.person.of.the.household',
                                'Gender.of.reference.person.of.the.household',
                                'Legal.marital.status.of.reference.person.of.the.household',
                                'Housing.arrangements',
                                'Type.of.living.quarter',
                                'Tenure.status.of.the.household',
                                'Type.of.private.household',
                                'Census.year',
                                'Number.of.members.in.the.household',
                                'Single.or.shared.occupancy', 
#                                 'Data.type', 
                                'Value')]
# get sole occupant data
soleOccupantData <- censusFAMData %>%
        filter(
        Age.class.of.reference.person.of.the.household == 'total',
        Gender.of.reference.person.of.the.household == 'total',
        Legal.marital.status.of.reference.person.of.the.household == 'total',
        Housing.arrangements == 'all items',
        Type.of.living.quarter == 'all items',
        Tenure.status.of.the.household == 'all items',
        Number.of.members.in.the.household == "1",
        Single.or.shared.occupancy == 'single occupancy',
        Census.year == censusYear
               )

soleOccupantData <- soleOccupantData[c("ITTER107","Value")]
colnames(soleOccupantData) <- c("ITTER107","soleOccupant_abs")


# get rid of the larger areas
soleOccupantData$ITTER107 <- as.numeric(as.character(soleOccupantData$ITTER107))
soleOccupantData <- na.omit(soleOccupantData)

#get the total number of households and join
soleOccupantData <- merge(totalHousholdsData, soleOccupantData, by = 'ITTER107')

# perc sole occupant
soleOccupantData$soleOccupant_perc <- round((soleOccupantData$soleOccupant_abs/soleOccupantData$total_households_abs)*100, 2)

# trim data
soleOccupantData <- soleOccupantData[c("ITTER107","soleOccupant_perc")]
# 
# soleOccupantData

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [16]:
#availability of shower and bathing facilities and drinking water
censusABITData <- censusABIT[, c('ITTER107', 
                                'Territory',
                                'Type.of.living.quarter',
                                'Number.of.rooms',
                                'Data.Type', 
                                'Goods.and.services',
                                'Census.year',
                                'Value')]

# get shower bath data
showerBathData <- censusABITData %>%
        filter(
        Type.of.living.quarter == "occupied conventional dwellings by resident persons",
        Number.of.rooms == 'total',
        Data.Type == 'number of conventional dwellings (absolute values)',
        Goods.and.services == 'no showers and no bathtubs ',
        Census.year == censusYear
               )


showerBathData <- showerBathData[c("ITTER107","Value")]
colnames(showerBathData) <- c("ITTER107","no_shower_bath_abs")

# get drinking water  data
drinkingWaterData <- censusABITData %>%
        filter(
        Type.of.living.quarter == "occupied conventional dwellings by resident persons",
        Number.of.rooms == 'total',
        Data.Type == 'number of conventional dwellings (absolute values)',
        Goods.and.services == 'drinkable water',
        Census.year == censusYear
               )

drinkingWaterData <- drinkingWaterData[c("ITTER107","Value")]
colnames(drinkingWaterData) <- c("ITTER107","drinking_water_abs")


# merge data
waterData <- merge(showerBathData, drinkingWaterData, by = 'ITTER107')

# get rid of the larger areas
waterData$ITTER107 <- as.numeric(as.character(waterData$ITTER107))
waterData <- na.omit(waterData)

# merge with total households data
waterData <- merge(waterData, totalHousholdsData , by = 'ITTER107')

# perc no bath shower
waterData$no_shower_bath_perc <- round((waterData$no_shower_bath_abs/waterData$total_households_abs)*100, 3)

# perc no water
waterData$drinking_water_perc <- round(((waterData$total_households_abs - waterData$drinking_water_abs)/waterData$total_households_abs)*100, 3)

# trim data
waterData <- waterData[c("ITTER107","no_shower_bath_perc", "drinking_water_perc")]

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [17]:
# Number of rooms
censusABITData <- censusABIT[, c('ITTER107', 
                                'Territory',
                                'Type.of.living.quarter',
                                'Number.of.rooms',
                                'Data.Type', 
                                'Goods.and.services',
                                'Census.year',
                                'Value')]

# get one room data
oneRoomData <- censusABITData %>%
        filter(
        Type.of.living.quarter == "occupied conventional dwellings by resident persons",
        Number.of.rooms == '1',
        Data.Type == 'number of conventional dwellings (absolute values)',
        Goods.and.services == 'all items',
        Census.year == censusYear
               )

oneRoomData <- oneRoomData[c("ITTER107","Value")]
colnames(oneRoomData) <- c("ITTER107","one_room_abs")

# get rid of the larger areas
oneRoomData$ITTER107 <- as.numeric(as.character(oneRoomData$ITTER107))
oneRoomData <- na.omit(oneRoomData)

# merge with total households data
oneRoomData <- merge(oneRoomData, totalHousholdsData , by = 'ITTER107')

# perc one room
oneRoomData$one_room_perc <- round((oneRoomData$one_room_abs/oneRoomData$total_households_abs)*100, 2)


# trim data
oneRoomData <- oneRoomData[c("ITTER107","one_room_perc")]


Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [18]:
# water source not from public water mains
censusABITData <- censusABIT[, c('ITTER107', 
                                'Territory',
                                'Type.of.living.quarter',
                                'Number.of.rooms',
                                'Data.Type', 
                                'Goods.and.services',
                                'Census.year',
                                'Value')]

# get one room data
waterQualityData <- censusABITData %>%
        filter(
        Type.of.living.quarter == "occupied conventional dwellings by resident persons",
        Number.of.rooms == 'total',
        Data.Type == 'number of conventional dwellings (absolute values)',
        Goods.and.services == 'from the public water mains',
        Census.year == censusYear
               )

waterQualityData <- waterQualityData[c("ITTER107","Value")]
colnames(waterQualityData) <- c("ITTER107","water_quality_abs")

# get rid of the larger areas
waterQualityData$ITTER107 <- as.numeric(as.character(waterQualityData$ITTER107))
waterQualityData <- na.omit(waterQualityData)

# merge with total households data
waterQualityData <- merge(waterQualityData, totalHousholdsData , by = 'ITTER107')

# perc no water
waterQualityData$water_quality_perc <- round(((waterQualityData$total_households_abs - waterQualityData$water_quality_abs)/waterQualityData$total_households_abs)*100, 3)

# trim data
waterQualityData <- waterQualityData[c("ITTER107","water_quality_perc")]


Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [19]:
# building age  DICA_ALLOGGI - Housing units - Full dataset
censusALLOGGIData <- censusALLOGGI[, c('ITTER107', 
                                'Territory',
                                'Goods.and.services',
                                'Number.of.rooms',
                                'Number.of.occupants',
                                'Useful.floor.space.groups..of.the.conventional.dwelling..square.metres.',
                                'Type.of.ownership',
                                'Number.of..conventional.dwellings',
                                'Number.of.dwellings.in.the.building',
                                'Occupancy.status.of.conventional.dwelling',    
                                'Census.year',
                                'Period.of.construction.of.the.building',                                
                                'Data.type', 
                                'Type.of.living.quarter',
                                'Type.of.building',
                                'Value')]

# building age data
# up to 1918	1919-1945	1946-1960	1961-1970	1971-1980	1981-1990	1991-2000	2001-2005	2006 or after
buildingAgeData <- censusALLOGGIData %>%
        filter(
        Goods.and.services == 'all items',
        Number.of.rooms == 'total',
        Number.of.occupants == 'total',
        Useful.floor.space.groups..of.the.conventional.dwelling..square.metres. == 'all items',
        Type.of.ownership == 'all items',
        Number.of..conventional.dwellings == 'total',
        Number.of.dwellings.in.the.building == 'total',
        Occupancy.status.of.conventional.dwelling == 'all items',
        Census.year == censusYear,
        Period.of.construction.of.the.building == 'up to 1918',                           
        Data.type == 'number of conventional dwellings (absolute values)',
        Type.of.living.quarter == 'all items',
        Type.of.building == 'residential buildings'
               )

buildingAgeData <- buildingAgeData[c("ITTER107","Value")]
colnames(buildingAgeData) <- c("ITTER107","pre_1918_abs")

# # get rid of the larger areas
buildingAgeData$ITTER107 <- as.numeric(as.character(buildingAgeData$ITTER107))
buildingAgeData <- na.omit(buildingAgeData)

# get the total number of buildings
buildingTotalData <- censusALLOGGIData %>%
        filter(
        Type.of.living.quarter == 'all items',
        Goods.and.services == 'all items',
        Number.of.rooms == 'total',
        Number.of.occupants == 'total',
        Useful.floor.space.groups..of.the.conventional.dwelling..square.metres. == 'all items',
        Type.of.ownership == 'all items',
        Number.of..conventional.dwellings == 'total',
        Number.of.dwellings.in.the.building == 'total',
        Occupancy.status.of.conventional.dwelling == 'all items',
        Census.year == censusYear,
        Period.of.construction.of.the.building == 'all items',                           
        Data.type == 'number of conventional dwellings (absolute values)',
        Type.of.building == 'residential buildings'
        
               )

buildingTotalData <- buildingTotalData[c("ITTER107","Value")]
colnames(buildingTotalData) <- c("ITTER107","total_abs")

# get rid of the larger areas
buildingTotalData$ITTER107 <- as.numeric(as.character(buildingTotalData$ITTER107))
buildingTotalData <- na.omit(buildingTotalData)


# ///
# # merge with total households data
buildingAgeData <- merge(buildingAgeData, buildingTotalData, by = 'ITTER107')

# # perc no water
buildingAgeData$pre_1918_perc <- round((buildingAgeData$pre_1918_abs/buildingAgeData$total_abs)*100, 2)

# trim data
buildingAgeData <- buildingAgeData[c("ITTER107","pre_1918_perc")]
# str(buildingAgeData)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [20]:
# average floor space DICA_ALLOGGI - Housing units - Full dataset
censusALLOGGIData <- censusALLOGGI[, c('ITTER107', 
                                'Territory',
                                'Goods.and.services',
                                'Number.of.rooms',
                                'Number.of.occupants',
                                'Useful.floor.space.groups..of.the.conventional.dwelling..square.metres.',
                                'Type.of.ownership',
                                'Number.of..conventional.dwellings',
                                'Number.of.dwellings.in.the.building',
                                'Occupancy.status.of.conventional.dwelling',    
                                'Census.year',
                                'Period.of.construction.of.the.building',   
                                'Type.of.living.quarter',
                                'Data.type', 
                                'Value')]

# avg floor space data
averageFloorSpaceData <- censusALLOGGIData %>%
        filter(
        Goods.and.services == 'all items',
        Number.of.rooms == 'total',
        Number.of.occupants == 'total',
        Useful.floor.space.groups..of.the.conventional.dwelling..square.metres. == 'all items',
        Type.of.ownership == 'all items',
        Number.of..conventional.dwellings == 'total',
        Number.of.dwellings.in.the.building == 'total',
        Occupancy.status.of.conventional.dwelling == 'all items',
        Census.year == censusYear,
        Period.of.construction.of.the.building == 'all items',                           
        Data.type == 'useful floor space by conventional dwellings occupant (square-metre) (average values)',
        Type.of.living.quarter == 'occupied conventional dwellings by resident persons'
               )

averageFloorSpaceData <- averageFloorSpaceData[c("ITTER107","Value")]
colnames(averageFloorSpaceData) <- c("ITTER107","avg_floorspace_sqm")

# get rid of the larger areas
averageFloorSpaceData$ITTER107 <- as.numeric(as.character(averageFloorSpaceData$ITTER107))
averageFloorSpaceData <- na.omit(averageFloorSpaceData)

# trim data
averageFloorSpaceData <- averageFloorSpaceData[c("ITTER107","avg_floorspace_sqm")]


Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [21]:
# number of floors - high rise DICA_EDIFICIRES - Residential buildings - Full dataset
censusEDIFICIREData<- censusEDIFICIRE[, c('ITTER107', 
                                'Territory',
                                'Percentage.class.of.occupied..conventional..dwellings',
                                'Number.of..conventional.dwellings',
                                'Number.of.dwellings.in.the.building',
                                'Number.of.staircase',
                                'Presence.of.lift',
                                'Number.of.floors.in.the.building..included.ground.floor.',
                                'Building.material',
                                'State.of.the.building',
                                'Period.of.construction.',
                                'Census.year',
                                'Data.Type', 
                                'Value')]

floorNumberData <- censusEDIFICIREData %>%
        filter(
        Percentage.class.of.occupied..conventional..dwellings == 'all items',
        Number.of..conventional.dwellings == 'total',
        Number.of.dwellings.in.the.building == 'total',
        Number.of.staircase == 'total',
        Presence.of.lift == 'total',
        Number.of.floors.in.the.building..included.ground.floor. == '4 and over',
        Building.material == 'all items',
        State.of.the.building == 'total',
        Period.of.construction. == 'all items',
        Census.year == censusYear,
        Data.Type == 'number of residential buildings (absolute values)'
        )

floorNumberData <- floorNumberData[c("ITTER107","Value")]
colnames(floorNumberData) <- c("ITTER107","floors_4_abs")

# get rid of the larger areas
floorNumberData$ITTER107 <- as.numeric(as.character(floorNumberData$ITTER107))
floorNumberData <- na.omit(floorNumberData)

# Get the total number of res buildings
totalNumberData <- censusEDIFICIREData %>%
        filter(
        Percentage.class.of.occupied..conventional..dwellings == 'all items',
        Number.of..conventional.dwellings == 'total',
        Number.of.dwellings.in.the.building == 'total',
        Number.of.staircase == 'total',
        Presence.of.lift == 'total',
        Number.of.floors.in.the.building..included.ground.floor. == 'total',
        Building.material == 'all items',
        State.of.the.building == 'total',
        Period.of.construction. == 'all items',
        Census.year == censusYear,
        Data.Type == 'number of residential buildings (absolute values)'
        )

totalNumberData <- totalNumberData[c("ITTER107","Value")]
colnames(totalNumberData) <- c("ITTER107","total_res_abs")

# get rid of the larger areas
totalNumberData$ITTER107 <- as.numeric(as.character(totalNumberData$ITTER107))
totalNumberData <- na.omit(totalNumberData)


# merge with total households data
floorNumberData <- merge(floorNumberData, totalNumberData , by = 'ITTER107')

# perc 4 floors or more
floorNumberData$floors_4_perc <- round((floorNumberData$floors_4_abs/floorNumberData$total_res_abs)*100, 2)

# trim data
floorNumberData <- floorNumberData[c("ITTER107","floors_4_perc")]


Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [22]:
#HOUSEHOLD DATA 

#merge all the data
householdData <- merge(totalHousholdsData, rentHousholdsData, by = 'ITTER107', all.x = TRUE)
householdData <- merge(householdData, singleParentData, by = 'ITTER107', all.x = TRUE)
householdData <- merge(householdData, soleOccupantData, by = 'ITTER107', all.x = TRUE)
householdData <- merge(householdData, waterQualityData, by = 'ITTER107', all.x = TRUE)
householdData <- merge(householdData, oneRoomData, by = 'ITTER107', all.x = TRUE)
householdData <- merge(householdData, waterData, by = 'ITTER107', all.x = TRUE)
householdData <- merge(householdData, buildingAgeData, by = 'ITTER107', all.x = TRUE)
householdData <- merge(householdData, averageFloorSpaceData, by = 'ITTER107', all.x = TRUE)
householdData <- merge(householdData, floorNumberData, by = 'ITTER107', all.x = TRUE)
# nrow(householdData)

#output the data as a csv
write.csv(householdData, file.path(pipelineDir, "households_data.csv"), row.names = FALSE)

# make it spatial 
householdDataSpatial <- merge(censusOA, householdData, by.x = 'PRO_COM', by.y = 'ITTER107', all.x = TRUE)

#output the data as a geojson
st_write(householdDataSpatial, file.path(pipelineDir, "oa_households.geojson"), delete_dsn = TRUE)

Deleting source `../../2_pipeline/Milan/1a_CensusData/2011/oa_households.geojson' using driver `GeoJSON'
Writing layer `oa_households' to data source 
  `../../2_pipeline/Milan/1a_CensusData/2011/oa_households.geojson' using driver `GeoJSON'
Writing 8092 features with 22 fields and geometry type Multi Polygon.


## Combine Households and People Data


In [23]:
# combine all the data together
allData <- merge(householdData, personsDataAll, by = 'ITTER107')
allData <- allData[, !names(allData) %in% c("Territory", "totalPresPop_abs", "total_households_abs")] 

#output the data as a csv
write.csv(allData, file.path(pipelineDir, "all_data.csv"), row.names = FALSE)

# make it spatial 
allDataSpatial <- merge(censusOA, allData, by.x = 'PRO_COM', by.y = 'ITTER107', all.x = TRUE)

#output the data as a geojson
st_write(allDataSpatial, file.path(pipelineDir, "oa_all.geojson"), delete_dsn = TRUE)

Deleting source `../../2_pipeline/Milan/1a_CensusData/2011/oa_all.geojson' using driver `GeoJSON'
Writing layer `oa_all' to data source 
  `../../2_pipeline/Milan/1a_CensusData/2011/oa_all.geojson' using driver `GeoJSON'
Writing 8092 features with 28 fields and geometry type Multi Polygon.


## Z-Scores

The raw data is not suitable for use within the vulnerabiltiy assessment. It needs to be standardised. Therefore, the data is converted to z-scores. Z-scores are:

>"A statistical measurement of a score's relationship to the mean (average value) in a group of scores. A Z-score of 0 means the score is the same as the mean (average value). A Z-score can be positive or negative, indicating whether it is above or below the mean and by how many standard deviations. Z-score standardisation represents the deviation of a raw score from its mean in standard deviation units." (Kazmierczak et al., 2015)

In [24]:
#Z scores

#Copy the data
allDataZ <- allData

#Calculate the z scores for each of the relevant columns - starting at the 2nd column
for(col in names(allDataZ)[2:ncol(allDataZ)]) {
#     print(col)
  allDataZ[paste0(col, "_Z")] = scale(allDataZ[col], )
}

#remove the original data to leave only the z scores
allDataZ <- allDataZ[-c(2:ncol(allData))]
# summary(personsDataZ)
head(allDataZ)

#reverse floor space
allDataZ$avg_floorspace_sqm_Z <-allDataZ$avg_floorspace_sqm_Z*-1 

#output the data as a csv
write.csv(allDataZ, file.path(pipelineDir, "all_data_z.csv"), row.names = FALSE)

,ITTER107,rent_households_perc_Z,singleParent_perc_Z,soleOccupant_perc_Z,water_quality_perc_Z,one_room_perc_Z,no_shower_bath_perc_Z,drinking_water_perc_Z,pre_1918_perc_Z,avg_floorspace_sqm_Z,floors_4_perc_Z,ageYoung_perc_Z,ageOld_perc_Z,popDen_Z,foreignCit_perc_Z,eduAttain_perc_Z,unemployed_perc_Z,ag_for_fi_employed_perc_Z
,<dbl>,"<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>"
1,1001,1.0270774,-1.0070983,0.26165318,0.11160762,0.21435000,-0.2135345,0.94674921,1.6024132,1.1232711,-0.4750655,-0.20036536,0.3483979,-0.15175905,0.4105886,-0.8162248,-0.466327832,-0.6035738
2,1002,1.1020597,-0.8658892,-0.87072229,0.09118114,1.33270047,-0.5802653,-0.38413133,-0.4206704,-1.1605725,-0.1266811,1.20179280,-1.0533790,-0.08606777,0.2454502,-0.0533230,-0.015262719,-0.7386059
3,1003,-0.8594074,-1.0750879,1.90937504,0.08653436,0.31735597,4.0038696,0.20400430,0.3995137,-0.3722235,1.1683004,-1.12053166,-1.0351741,-0.45418893,-1.0971963,-0.9729449,-1.022217937,0.6210282
4,1004,1.6201187,-0.2801330,-0.09654722,-0.34387367,0.50865276,-0.1664248,-0.57691574,0.6649003,0.4567271,-0.6618637,-0.34934467,-0.2220222,-0.22843052,0.1425379,-1.0379985,-0.545740704,-0.5279092
5,1005,-0.5986737,1.2783970,1.66903412,-0.31066855,1.94337869,0.6670542,-0.07586714,1.7566168,2.1535383,-0.3604654,-0.40192560,0.3362613,-0.31967369,-0.9296646,-1.8097712,-0.004144916,-0.6350037
6,1006,0.4118823,-0.1859936,-0.12196790,-0.39218082,-0.03580734,-0.1236637,-0.37768927,-0.2927261,0.5210131,-0.4257875,-0.07767652,-0.3919345,0.08790654,-0.2954377,-1.4371913,-0.475857377,-0.7828406


##  Milan 

In [25]:
# get the milan province
censusOAMilan <- censusOA %>% filter(COD_PROV == 15)

#get the milan data
milanData <- filter(allData, ITTER107 %in% censusOAMilan$PRO_COM)

#output the data as a csv
write.csv(milanData, file.path(pipelineDir, "oa_person_household_milan_data.csv"), row.names = FALSE)

# calculate milan Z
#Copy the data
milanDataZ <- milanData

#Calculate the z scores for each of the relevant columns - starting at the 2nd column
for(col in names(milanDataZ)[2:ncol(milanDataZ)]) {
#     print(col)
  milanDataZ[paste0(col, "_Z")] = scale(milanDataZ[col], )
}

#reverse floor space
milanDataZ$avg_floorspace_sqm_Z <-milanDataZ$avg_floorspace_sqm_Z*-1 

#remove the original data to leave only the z scores
#output the data as a csv
milanDataZ <- milanDataZ[-c(2:ncol(milanData))]
write.csv(milanDataZ, file.path(pipelineDir, "oa_person_household_milan_z_data.csv"), row.names = FALSE)

**END**